In [4]:
import PySpin
import sys
import time

NUM_IMAGES = 10  # Number of images to capture
TRIGGER_SOURCE = 'Line3'  # Trigger source to Line3 (green wire)

class TriggerType:
    HARDWARE = 2

CHOSEN_TRIGGER = TriggerType.HARDWARE

def configure_trigger(cam):
    """
    Configures the camera to use a hardware trigger on Line3.
    """
    try:
        # Ensure trigger mode off
        nodemap = cam.GetNodeMap()
        node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
        node_trigger_mode_off = node_trigger_mode.GetEntryByName('Off')
        node_trigger_mode.SetIntValue(node_trigger_mode_off.GetValue())

        # Set TriggerSelector to FrameStart
        node_trigger_selector = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSelector'))
        node_trigger_selector_framestart = node_trigger_selector.GetEntryByName('FrameStart')
        node_trigger_selector.SetIntValue(node_trigger_selector_framestart.GetValue())

        # Select trigger source
        node_trigger_source = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSource'))
        node_trigger_source_hardware = node_trigger_source.GetEntryByName(TRIGGER_SOURCE)
        node_trigger_source.SetIntValue(node_trigger_source_hardware.GetValue())

        # Set TriggerActivation to RisingEdge
        node_trigger_activation = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerActivation'))
        node_trigger_activation_risingedge = node_trigger_activation.GetEntryByName('RisingEdge')
        node_trigger_activation.SetIntValue(node_trigger_activation_risingedge.GetValue())

        # Turn trigger mode on
        node_trigger_mode_on = node_trigger_mode.GetEntryByName('On')
        node_trigger_mode.SetIntValue(node_trigger_mode_on.GetValue())

        print(f'Trigger configured for camera {cam.GetUniqueID()} to hardware (Line3) with rising edge activation.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def configure_exposure(cam, exposure_time):
    """
    Configures exposure settings for the camera.
    """
    try:
        nodemap = cam.GetNodeMap()

        # Turn off auto exposure
        node_exposure_auto = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureAuto'))
        if PySpin.IsAvailable(node_exposure_auto) and PySpin.IsWritable(node_exposure_auto):
            node_exposure_auto_off = node_exposure_auto.GetEntryByName('Off')
            node_exposure_auto.SetIntValue(node_exposure_auto_off.GetValue())
            print(f'Auto exposure turned off for camera {cam.GetUniqueID()}.')

        # Set exposure time
        node_exposure = PySpin.CFloatPtr(nodemap.GetNode('ExposureTime'))
        if PySpin.IsAvailable(node_exposure) and PySpin.IsWritable(node_exposure):
            exposure_to_set = min(node_exposure.GetMax(), max(node_exposure.GetMin(), exposure_time))
            node_exposure.SetValue(exposure_to_set)
            print(f'Exposure set to {exposure_to_set} microseconds for camera {cam.GetUniqueID()}.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def grab_next_image_by_trigger(cam):
    """
    Waits for hardware trigger to capture image.
    """
    try:
        # Wait for hardware trigger to capture image
        print(f'Waiting for hardware trigger for camera {cam.GetUniqueID()}...')
        image_result = cam.GetNextImage()

        if image_result.IsIncomplete():
            print(f'Image incomplete for camera {cam.GetUniqueID()} with image status {image_result.GetImageStatus()} ...')
        else:
            print(f'Image grabbed successfully for camera {cam.GetUniqueID()}.')
            return image_result

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return None

    return None

def acquire_images(cams):
    """
    Acquires and saves images from both cameras.
    """
    try:
        # Set acquisition mode to continuous for both cameras
        for cam in cams:
            nodemap = cam.GetNodeMap()
            node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
            node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
            node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())
            cam.BeginAcquisition()

        print('Acquiring images...')

        for i in range(NUM_IMAGES):
            images = []

            for cam in cams:
                image_result = grab_next_image_by_trigger(cam)
                if image_result:
                    images.append((cam, image_result))
                else:
                    print(f'Failed to grab image for camera {cam.GetUniqueID()}.')

            for cam, image_result in images:
                device_serial_number = ''
                nodemap_tldevice = cam.GetTLDeviceNodeMap()
                node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
                if PySpin.IsReadable(node_device_serial_number):
                    device_serial_number = node_device_serial_number.GetValue()

                filename = f'Trigger-{device_serial_number}-{i}.jpg'
                image_result.Save(filename)
                print(f'Image saved at {filename}')
                image_result.Release()

        # End acquisition for both cameras
        for cam in cams:
            cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def reset_trigger(cams):
    """
    Returns the cameras to a normal state by turning off trigger mode.
    """
    try:
        for cam in cams:
            nodemap = cam.GetNodeMap()
            node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
            node_trigger_mode_off = node_trigger_mode.GetEntryByName('Off')
            node_trigger_mode.SetIntValue(node_trigger_mode_off.GetValue())
            print(f'Trigger mode disabled for camera {cam.GetUniqueID()}.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def print_device_info(cam):
    """
    Prints the device information of the camera.
    """
    print(f'*** DEVICE INFORMATION FOR CAMERA {cam.GetUniqueID()} ***\n')
    try:
        nodemap = cam.GetTLDeviceNodeMap()
        node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))
        if PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print(f'{node_feature.GetName()}: {node_feature.ToString() if PySpin.IsReadable(node_feature) else "Node not readable"}')
        else:
            print('Device control information not readable.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def run_cameras(cams):
    """
    Runs the example on the provided cameras.
    """
    try:
        result = True

        # Initialize cameras and print device information
        for cam in cams:
            print_device_info(cam)
            cam.Init()

        # Configure trigger and exposure for both cameras
        for cam in cams:
            if not configure_trigger(cam):
                return False
            if not configure_exposure(cam, 9000):  # Set exposure to 9000 microseconds
                return False

        # Acquire images from both cameras
        result &= acquire_images(cams)

        # Reset trigger for both cameras
        result &= reset_trigger(cams)

        # Deinitialize cameras
        for cam in cams:
            cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

def main():
    """
    Main entry point for the example.
    """
    result = True

    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print(f'Library version: {version.major}.{version.minor}.{version.type}.{version.build}')

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()
    num_cameras = cam_list.GetSize()

    print(f'Number of cameras detected: {num_cameras}')

    # Finish if there are not exactly two cameras
    if num_cameras != 2:
        cam_list.Clear()
        system.ReleaseInstance()
        print('Exactly 2 cameras are required.')
        return False

    # Get camera references
    cams = [cam_list[0], cam_list[1]]

    # Run example on both cameras
    result &= run_cameras(cams)

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)


Library version: 4.0.0.116
Number of cameras detected: 2
*** DEVICE INFORMATION FOR CAMERA USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000_0 ***

DeviceID: USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000_0
DeviceSerialNumber: 16290112
DeviceUserID: 
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-13Y3M
DeviceVersion: 1808.0.120.0
DeviceBootloaderVersion: Node not readable
DeviceType: USB3Vision
DeviceDisplayName: FLIR Blackfly S BFS-U3-13Y3M
DeviceAccessStatus: OpenReadWrite
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000
DeviceLocation: 0000.0014.0000.021.000.000.000.000.000
DeviceCurrentSpeed: SuperSpeed
DeviceU3VProtocol: 1
DevicePortId: 21
GenICamXMLLocation: Device
GenICamXMLPath: 
GUIXMLLocation: Device
GUIXMLPath: Input.xml
*** DEVICE INFORMATION FOR CAMERA USB\VID_1E10&PID_4000&MI_00\6&B540B48&0&0000_0 ***

DeviceID: USB\VID_1E10&PID_4000&MI_00\6&B540B48&0&0000_0
DeviceSerialNumber: 1629010

SystemExit: 0

Manually set image size and resolution

In [27]:
import PySpin
import sys
import time

# Configuration parameters (adjust as needed)
IMAGE_WIDTH = 1280  # Image width in pixels
IMAGE_HEIGHT = 650  # Image height in pixels
EXPOSURE_TIME = 9000  # Exposure time in microseconds
NUM_IMAGES = 10  # Number of images to capture
TRIGGER_SOURCE = 'Line3'  # Trigger source to Line3 (green wire)

class TriggerType:
    HARDWARE = 2

CHOSEN_TRIGGER = TriggerType.HARDWARE

def configure_trigger(cam):
    """
    Configures the camera to use a hardware trigger on Line3.
    """
    try:
        # Ensure trigger mode off
        nodemap = cam.GetNodeMap()
        node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
        node_trigger_mode_off = node_trigger_mode.GetEntryByName('Off')
        node_trigger_mode.SetIntValue(node_trigger_mode_off.GetValue())

        # Set TriggerSelector to FrameStart
        node_trigger_selector = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSelector'))
        node_trigger_selector_framestart = node_trigger_selector.GetEntryByName('FrameStart')
        node_trigger_selector.SetIntValue(node_trigger_selector_framestart.GetValue())

        # Select trigger source
        node_trigger_source = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerSource'))
        node_trigger_source_hardware = node_trigger_source.GetEntryByName(TRIGGER_SOURCE)
        node_trigger_source.SetIntValue(node_trigger_source_hardware.GetValue())

        # Set TriggerActivation to RisingEdge
        node_trigger_activation = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerActivation'))
        node_trigger_activation_risingedge = node_trigger_activation.GetEntryByName('RisingEdge')
        node_trigger_activation.SetIntValue(node_trigger_activation_risingedge.GetValue())

        # Turn trigger mode on
        node_trigger_mode_on = node_trigger_mode.GetEntryByName('On')
        node_trigger_mode.SetIntValue(node_trigger_mode_on.GetValue())

        print(f'Trigger configured for camera {cam.GetUniqueID()} to hardware (Line3) with rising edge activation.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def configure_exposure(cam, exposure_time):
    """
    Configures exposure settings for the camera.
    """
    try:
        nodemap = cam.GetNodeMap()

        # Turn off auto exposure
        node_exposure_auto = PySpin.CEnumerationPtr(nodemap.GetNode('ExposureAuto'))
        if PySpin.IsAvailable(node_exposure_auto) and PySpin.IsWritable(node_exposure_auto):
            node_exposure_auto_off = node_exposure_auto.GetEntryByName('Off')
            node_exposure_auto.SetIntValue(node_exposure_auto_off.GetValue())
            print(f'Auto exposure turned off for camera {cam.GetUniqueID()}.')

        # Set exposure time
        node_exposure = PySpin.CFloatPtr(nodemap.GetNode('ExposureTime'))
        if PySpin.IsAvailable(node_exposure) and PySpin.IsWritable(node_exposure):
            exposure_to_set = min(node_exposure.GetMax(), max(node_exposure.GetMin(), exposure_time))
            node_exposure.SetValue(exposure_to_set)
            print(f'Exposure set to {exposure_to_set} microseconds for camera {cam.GetUniqueID()}.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def configure_image_format(cam, width, height):
    """
    Configures the image format (width and height) for the camera.
    """
    try:
        nodemap = cam.GetNodeMap()

        # Set width
        node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
        if PySpin.IsAvailable(node_width) and PySpin.IsWritable(node_width):
            max_width = node_width.GetMax()
            node_width.SetValue(min(max_width, width))
            print(f'Width set to {width} pixels for camera {cam.GetUniqueID()}.')

        # Set height
        node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
        if PySpin.IsAvailable(node_height) and PySpin.IsWritable(node_height):
            max_height = node_height.GetMax()
            node_height.SetValue(min(max_height, height))
            print(f'Height set to {height} pixels for camera {cam.GetUniqueID()}.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def grab_next_image_by_trigger(cam):
    """
    Waits for hardware trigger to capture image.
    """
    try:
        # Wait for hardware trigger to capture image
        print(f'Waiting for hardware trigger for camera {cam.GetUniqueID()}...')
        image_result = cam.GetNextImage()

        if image_result.IsIncomplete():
            print(f'Image incomplete for camera {cam.GetUniqueID()} with image status {image_result.GetImageStatus()} ...')
        else:
            print(f'Image grabbed successfully for camera {cam.GetUniqueID()}.')
            return image_result

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return None

    return None

def acquire_images(cams):
    """
    Acquires and saves images from both cameras.
    """
    try:
        # Set acquisition mode to continuous for both cameras
        for cam in cams:
            nodemap = cam.GetNodeMap()
            node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
            node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
            node_acquisition_mode.SetIntValue(node_acquisition_mode_continuous.GetValue())
            cam.BeginAcquisition()

        print('Acquiring images...')

        for i in range(NUM_IMAGES):
            images = []

            for cam in cams:
                image_result = grab_next_image_by_trigger(cam)
                if image_result:
                    images.append((cam, image_result))
                else:
                    print(f'Failed to grab image for camera {cam.GetUniqueID()}.')

            for cam, image_result in images:
                device_serial_number = ''
                nodemap_tldevice = cam.GetTLDeviceNodeMap()
                node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
                if PySpin.IsReadable(node_device_serial_number):
                    device_serial_number = node_device_serial_number.GetValue()

                filename = f'Trigger-{device_serial_number}-{i}.jpg'
                image_result.Save(filename)
                print(f'Image saved at {filename}')
                image_result.Release()

        # End acquisition for both cameras
        for cam in cams:
            cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def reset_trigger(cams):
    """
    Returns the cameras to a normal state by turning off trigger mode.
    """
    try:
        for cam in cams:
            nodemap = cam.GetNodeMap()
            node_trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))
            node_trigger_mode_off = node_trigger_mode.GetEntryByName('Off')
            node_trigger_mode.SetIntValue(node_trigger_mode_off.GetValue())
            print(f'Trigger mode disabled for camera {cam.GetUniqueID()}.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def print_device_info(cam):
    """
    Prints the device information of the camera.
    """
    print(f'*** DEVICE INFORMATION FOR CAMERA {cam.GetUniqueID()} ***\n')
    try:
        nodemap = cam.GetTLDeviceNodeMap()
        node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))
        if PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print(f'{node_feature.GetName()}: {node_feature.ToString() if PySpin.IsReadable(node_feature) else "Node not readable"}')
        else:
            print('Device control information not readable.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True

def run_cameras(cams):
    """
    Runs the example on the provided cameras.
    """
    try:
        result = True

        # Initialize cameras and print device information
        for cam in cams:
            print_device_info(cam)
            cam.Init()

        # Configure trigger, exposure, and image format for both cameras
        for cam in cams:
            if not configure_trigger(cam):
                return False
            if not configure_exposure(cam, EXPOSURE_TIME):
                return False
            if not configure_image_format(cam, IMAGE_WIDTH, IMAGE_HEIGHT):
                return False

        # Acquire images from both cameras
        result &= acquire_images(cams)

        # Reset trigger for both cameras
        result &= reset_trigger(cams)

        # Deinitialize cameras
        for cam in cams:
            cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

def main():
    """
    Main entry point for the example.
    """
    result = True

    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print(f'Library version: {version.major}.{version.minor}.{version.type}.{version.build}')

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()
    num_cameras = cam_list.GetSize()

    print(f'Number of cameras detected: {num_cameras}')

    # Finish if there are not exactly two cameras
    if num_cameras != 2:
        cam_list.Clear()
        system.ReleaseInstance()
        print('Exactly 2 cameras are required.')
        return False

    # Get camera references
    cams = [cam_list[0], cam_list[1]]

    # Run example on both cameras
    result &= run_cameras(cams)

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)

Library version: 4.0.0.116
Number of cameras detected: 2
*** DEVICE INFORMATION FOR CAMERA USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000_0 ***

DeviceID: USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000_0
DeviceSerialNumber: 16290112
DeviceUserID: 
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-13Y3M
DeviceVersion: 1808.0.120.0
DeviceBootloaderVersion: Node not readable
DeviceType: USB3Vision
DeviceDisplayName: FLIR Blackfly S BFS-U3-13Y3M
DeviceAccessStatus: OpenReadWrite
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&2A5D0DF9&0&0000
DeviceLocation: 0000.0014.0000.021.000.000.000.000.000
DeviceCurrentSpeed: SuperSpeed
DeviceU3VProtocol: 1
DevicePortId: 21
GenICamXMLLocation: Device
GenICamXMLPath: 
GUIXMLLocation: Device
GUIXMLPath: Input.xml
*** DEVICE INFORMATION FOR CAMERA USB\VID_1E10&PID_4000&MI_00\6&B540B48&0&0000_0 ***

DeviceID: USB\VID_1E10&PID_4000&MI_00\6&B540B48&0&0000_0
DeviceSerialNumber: 1629010

SystemExit: 0